---
title: Contributor Metrics -- pyOpenSci
subtitle: More
license:
  code: BSD-3
---

In [8]:
import os
import warnings
from datetime import datetime
from pathlib import Path

import altair as alt
import pandas as pd

# This is a local module that stores the plot theme
from pyosmetrics.plot_theme import load_poppins_font, register_and_enable_poppins_theme

pd.options.mode.chained_assignment = None
pd.options.future.infer_string = True

# Suppress all warnings
warnings.filterwarnings("ignore")

# Load the & register Poppins theme
load_poppins_font()
register_and_enable_poppins_theme()

In [9]:
# Get the current notebook directory
current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
data_dir = parent_dir / "_data"
all_files = list(data_dir.glob("*"))
dfs = [pd.read_csv(file, parse_dates=["item_opened_by"]) for file in all_files]
df = pd.concat(dfs, ignore_index=True)

## All contributions to date 

In [10]:
df["quarter"] = df["item_opened_by"].dt.to_period("Q")

# Group by quarter and contrib_type (staff vs non-staff)
df_quarters = df.groupby(["quarter", "contrib_type"]).size().reset_index(name="count")
df_quarters["quarter"] = df_quarters["quarter"].astype(str)

# Create a new column for filtered labels
df_quarters["quarter_label"] = df_quarters["quarter"]
df_quarters["year"] = df_quarters["quarter"].str[:4]
df_quarters["qtr"] = df_quarters["quarter"].str[-2:]
df_quarters.reset_index(inplace=True)

,index,quarter,contrib_type,count,quarter_label,year,qtr
0,0,2018Q4,staff,2,2018Q4,2018,Q4
1,1,2019Q1,contributor,42,2019Q1,2019,Q1
2,2,2019Q1,staff,1,2019Q1,2019,Q1
3,3,2019Q2,contributor,47,2019Q2,2019,Q2
4,4,2019Q2,staff,34,2019Q2,2019,Q2
5,5,2019Q3,contributor,24,2019Q3,2019,Q3
6,6,2019Q3,staff,10,2019Q3,2019,Q3
7,7,2019Q4,contributor,39,2019Q4,2019,Q4


In [12]:
# Create the Altair chart
chart = (
    alt.Chart(df_quarters)
    .mark_bar()
    .encode(
        x=alt.X(
            "quarter:O",
            title="Quarter",
            axis=alt.Axis(labelExpr='datum.index % 3 == 0 ? datum.label : ""')),
        y=alt.Y("count:Q", stack=True, title="Count"),
        color=alt.Color(
            "contrib_type:N",
            title="Contributor Type (Staff vs Non-Staff)",
            scale=alt.Scale(range=["#81c0aa", "#735fab"]),
        ),
        tooltip=["quarter", "contrib_type", "count"],
    )
    .properties(title="Staff vs. Non Staff Contributor by Quarter")
    .configure_legend(
        orient="top",
        title=None,
        legendX=-550,
        legendY=-30,
    )
    .interactive()
)

chart

alt.Chart(...)